# Обучение без учителя

In [15]:
import numpy as np
import pandas as pd

### Task 1

Данные о посещениях заведений, о регистрации пользователя в заведениях, о геолокации заведений [здесь](https://files.sberdisk.ru/s/cW1tzZwkejcCvTP) \
Очистим данные от пустых записей и выведем количество записей после очистки

In [16]:
df_visit = pd.read_csv('../datasets/checkins.dat', sep='|', skipinitialspace=True, na_values=' ', low_memory=False)
                                            # разделитель | и пробелы после | убрали, пустые значения заменили на NAN
df_visit.columns = df_visit.columns.str.strip()  # удалили пробелы в названии столбцов
#df_visit.columns
df_visit.dropna(inplace=True) # убрали пустые строки
print('Количество записей после очистки -', df_visit.shape[0])
#df_visit

Количество записей после очистки - 396634


### Task 2

Данные содержат записи о заведениях во всём мире. С помощью геолокаций и библиотеки *Reverse Geocoder*, создадим столбец с названием страны для каждой геопозиции \
Выведем **Название второй страны по количеству записей**

In [17]:
import reverse_geocoder as rg
import pycountry

coordinates = list(zip(df_visit['latitude'], df_visit['longitude'])) # список кортежей (широта, долгота)
#coordinates
results =  rg.search(coordinates) # список локаций, название стран по ключу 'cc'
#results
df_visit['country'] = [country['cc'] for country in results] # добавляем колонку с названием страны в df_visit
#df_visit

country_val = df_visit['country'].value_counts() # серия упорядоченная по количеству упоминаний страны в столбце 'country'
#country_val
print('Название второй страны по количеству записей:', pycountry.countries.get(alpha_2=country_val.index[1]).name)

Название второй страны по количеству записей: Indonesia


### Task 3

Далее работаем с американскими геолокациями \
Чтобы уменьшить количество геолокаций, оставим 50 самых часто встречаеющихся заведений **venue** \
Выведем количество записей, оставшихся после очисток

In [18]:
df_us = df_visit[df_visit['country']=="US"] # отсортировываем только локации US
#df_us
us_50 = df_us['venue_id'].value_counts().head(50) # 50 самых часто встречаемых заведений 
df_us_50 = df_us[df_us['venue_id'].isin(us_50.index)] # отсортировываем df_us по заведениям из us_50

print('Количество оставшихся локаций -', df_us_50.shape[0])

Количество оставшихся локаций - 162099


### Task 4

Задача кластеризации \
Используем алгоритм метода *Mean Shift* из модуля *sklearn.cluster* для кластеризации локаций \
Выведем количество кластеров, полученных в результате кластеризации

In [19]:
from sklearn.cluster import MeanShift

ms = MeanShift(bandwidth=0.1, bin_seeding=True) # кластеризация Mean Shift
ms = ms.fit(df_us_50[['latitude', 'longitude']]) # используем кластеризацию для локаций из df_us_50
ms_center = ms.cluster_centers_ # центры кластеров
#ms_center

print('Количество кластеров -', ms_center.shape[0])

Количество кластеров - 2846


### Task 5

Найдём места для установки банеров - это центры кластеров, которые ближе всего расположены к офисам продаж компании и для каждого офиса найдём 5 самых ближайших к нему центров кластеров \
В компании 11 офисов, т.е. получится 55 мест для установки банеров \
Выведем координаты банера, ближайшего к офису компании

In [20]:
from sklearn.neighbors import KDTree

df_office = pd.read_csv('../datasets/offices.csv') # df с координатами офисов
df_center = pd.DataFrame(data=ms_center, columns=['latitude', 'longitude']) # df с координатами центров кластеров
#df_center

kdt = KDTree(df_center[['latitude', 'longitude']], metric='euclidean') # используем класс KDTree для поиска ближайш. соседей из df_center
distance, index_ = kdt.query(df_office[['latitude', 'longitude']], k=5) # два множ-та по 5 центров кластеров с ближайш. дистанциями и индексами к каж. оф.
#distance, index

#min_office = np.unravel_index(distance.argmin(), distance.shape)[0]+1 # оф. возле которого ближайший банер
min_center = index_[np.unravel_index(distance.argmin(), distance.shape)] # индекс ближайшего к оф. центра кластера(банера)
lat_min, long_min = df_center[['latitude', 'longitude']].loc[min_center]

print('Координаты установки баннера, который ближе всего находится к офису компании: ({}, {})'.format(lat_min, long_min))
#df_center, distance, index_

Координаты установки баннера, который ближе всего находится к офису компании: (32.78531777579914, -79.92474241187222)


### Task 6

Используя функцию *scatter_mapbox* отметим точки установки банеров на карте

In [21]:
import plotly.express as px

df_banner = df_center.iloc[np.ravel(index_).tolist()].copy() # выбираем из df_center только те которые вышли в index_ т.е. по 5 ближайш. к каж. оф.
#df_banner.index.rename('center', inplace= True ) # переименовываем столбец индексов в center т.к. это индексы баннеров(центры кластеров)
str_office = [str(i+1) for i in range(10) for j in range(5)] # создаём строчку соответствия № офисов
df_banner['office'] = str_office # добавляем в df_banner столбец с № соответствующих офисов
#df_banner

fig_ = px.scatter_mapbox(df_banner, lat='latitude', lon='longitude', color=df_banner['office'], mapbox_style='open-street-map',
                         zoom=3, width=950, height=550, title='Карта расположения баннеров и офисов')
fig_.show()